In [2]:
import numpy as np
from osgeo import gdal
import datetime

In [3]:
data_in = gdal.Open(r"ml_2015_input.dat")
im_width = data_in.RasterXSize
im_height = data_in.RasterYSize
im_band = data_in.RasterCount
im_geotrans = data_in.GetGeoTransform()
im_proj = data_in.GetProjection()

In [4]:
zheight = 1000  # height where wind speed was measured, cm
z0 = 7.9  # roughness length, cm
A = 0.026
efld = 5.6  # e-folding of u*s/u* vs x/h
U = 0.29  # value of u*s/u* at x=0
minsize = 0.05  # minimum scaled gap size
maxsize = 100  # maximum scaled gap size
# ust = 25  # unvegetated soil threshold shear velocity, u*t, cm/s
p = 0.00129   # g/cm3
g = 980  # cm/s2

In [5]:
data_read = data_in.ReadAsArray(0, 0, im_width, im_height)
gap25 = data_read[0, :, :]
gap50 = data_read[1, :, :]
gap100 = data_read[2, :, :]
gap200 = data_read[3, :, :]
gap250 = data_read[4, :, :]
bare = data_read[5, :, :]
veg = data_read[6, :, :]
plth = data_read[7, :, :]
wind = data_read[8, :, :]
sand = data_read[9, :, :]
clay = data_read[10, :, :]
rock = data_read[11, :, :]

In [7]:
mask_sum=np.sum(data_read,axis=0)
mask = np.where(mask_sum > 1)
gap25_mask = gap25[mask]
gap50_mask = gap50[mask]
gap100_mask = gap100[mask]
gap200_mask = gap200[mask]
gap250_mask = gap250[mask]
bare_mask = bare[mask]
veg_mask = veg[mask]
plth_mask = plth[mask]
wind_mask = wind[mask]
sand_mask = sand[mask]
clay_mask = clay[mask]
rock_mask = rock[mask]

In [8]:
# build wind pdf
wind1 = 0.1
wind2 = 0.2
wind3 = 0.3
wind4 = 0.3
wind5 = 0.1
u = np.array([1, 2, 3, 4, 5]) * 100  # windspeed
ustar_hist = np.array([wind1, wind2, wind3, wind4, wind5])  # frequency
ustar = (u * 0.4) / np.log10(zheight / z0)
ustar_hist = ustar_hist / np.sum(ustar_hist)

In [9]:
# build gap pdf
gap_sum = gap25_mask + gap50_mask + gap100_mask + gap200_mask + gap250_mask
gap25_dist = gap25_mask / gap_sum
gap50_dist = gap50_mask / gap_sum
gap100_dist = gap100_mask / gap_sum
gap200_dist = gap200_mask / gap_sum
gap250_dist = gap250_mask / gap_sum
gap_total = np.array([gap25_mask, gap50_mask, gap100_mask,
                      gap200_mask, gap250_mask])
gap_dist = np.array([gap25_dist, gap50_dist, gap100_dist,
                     gap200_dist, gap250_dist])

In [10]:
# gapsize and non_smoothpd
x = np.array([37.5/plth_mask, 75/plth_mask, 150/plth_mask,
              225/plth_mask, 500/plth_mask])   # sclaed gap size
pg = gap_dist  # probability distribution of the size of unvegetated gap
pd = pg / x
for i in range(x.shape[0]):
    temp = np.where((x[i, :] < minsize) | (x[i, :] > maxsize))
    temp = np.array(temp)
    if temp.shape != 0:
        pd[i, temp] = 0

In [14]:
# wemo5_core
n_windspeeds = ustar.size
n_x = gap_dist.shape[0]
q = np.zeros((n_x, n_windspeeds))
q_array = np.zeros((n_x))
Ftot_veg = np.zeros((gap_dist.shape[1]))
Ftot_bare = np.zeros((gap_dist.shape[1]))
ust = np.zeros((gap_dist.shape[1]))
f = np.zeros((gap_dist.shape[1]))
qtot = np.zeros((gap_dist.shape[1]))
k_con = np.zeros((gap_dist.shape[1]))
uss_mean = np.zeros((gap_dist.shape[1]))
Us_mean = np.zeros((gap_dist.shape[1]))

for i in range(gap_dist.shape[1]):
    Us = U + ((1 - U) * (1 - np.exp(-(x[:, i] / efld))))
    Us_mean[i] = np.mean(Us)
    if clay_mask[i] > 20:
        ust[i] = 400 + 2.4 * rock_mask[i]
    else:
        ust[i] = 165.7 - 139 * (sand_mask[i] + clay_mask[i]) / 100 + 2.4 * rock_mask[i]
    for k in range(0, n_windspeeds):
        uss = ustar[k] * Us
        q_array = A * p / g * uss * ((uss) ** 2 - ust[i] ** 2)
        q[:, k] = q_array * ustar_hist[k]
    for j in range(0, n_x):
        q[j, :] = q[j, :] * pd[j, i]
    uss_mean[i] = np.mean(uss)

    qtot[i] = A * np.sum(q)
    if qtot[i] < 0:
        qtot[i] = 0
    qtot[i] = qtot[i] * 3600 * 24
    if clay_mask[i] < 20:
        k_con[i] = np.exp(0.134 * clay_mask[i] - 6)
    else:
        k_con[i] = 0.005

    f[i] = qtot[i] * k_con[i]
    Ftot_veg[i] = f[i] * (100 - veg_mask[i]) / 100 * 30
    # Ftot_bare[i] = f[i] * bare_mask[i] / 100

In [15]:
# output to a image
myarray = np.array([ust, qtot, k_con, f, uss_mean, Us_mean, Ftot_veg])
mylist = ["ust", "qtot", "k_con", "f", "uss_mean", "Us_mean", "Ftot_veg"]
temp = gap25.reshape(im_width*im_height)
out_image = np.zeros((len(mylist), im_height, im_width))
for s in range(len(mylist)):
    new = np.zeros(im_width*im_height)
    new[mask] = myarray[s, :]
    out_image[s, :, :] = new.reshape(im_height, im_width)

In [26]:
driver = gdal.GetDriverByName("ENVI")
dataset = driver.Create(r"ml_2015_result.dat", im_width,
                        im_height, len(mylist), gdal.GDT_Float32)
dataset.SetGeoTransform(im_geotrans)
dataset.SetProjection(im_proj)
# dataset.GetRasterBand(1).WriteArray(out_image)
for s, image in enumerate(out_image, 1):
    dataset.GetRasterBand(s).WriteArray(image)
    dataset.GetRasterBand(s).SetDescription(mylist[s-1])

del dataset